# Notebook 9: Avaliação de Modelos e Definição de Baseline

## Objetivo
Aprender a avaliar se o seu modelo de detecção de anomalias "funciona" e como definir o ponto de corte (threshold).

## O Desafio
Em detecção de anomalias **não-supervisionada**, você não tem labels (não sabe o que é fraude). Como saber se o modelo é bom?
1.  **Avaliação Interna**: Estabilidade, consenso entre modelos.
2.  **Avaliação Externa (com Labels)**: Usando um pequeno conjunto anotado manualmente por auditores.

Neste notebook, assumiremos que auditamos uma amostra piloto e temos os labels verdadeiros para validar o modelo.

In [ ]:
!pip install -q pyod pandas matplotlib seaborn scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Gerando Dados e Treinando Modelo
Usaremos iForest como exemplo.

In [ ]:
X_train, X_test, y_train, y_test = generate_data(n_train=1000, n_test=500, contamination=0.1, random_state=42)

clf = IForest(random_state=42)
clf.fit(X_train)

# Obtendo scores de teste
y_scores = clf.decision_function(X_test)

## 2. A Curva ROC e AUC
A métrica AUC (Area Under Curve) diz o quão bem o modelo separa as classes, INDEPENDENTE do threshold escolhido.
- AUC = 0.5: Aleatório
- AUC = 1.0: Perfeito

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)')
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

## 3. Curva Precision-Recall
Em detecção de fraude, os dados são muito desbalanceados (poucas fraudes). A curva ROC pode ser otimista demais. A curva Precision-Recall é mais honesta.

In [ ]:
precision, recall, _ = precision_recall_curve(y_test, y_scores)
avg_precision = average_precision_score(y_test, y_scores)

plt.figure()
plt.plot(recall, precision, color='green', lw=2, label=f'Precision-Recall (avg = {avg_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend(loc="lower left")
plt.show()

## 4. Precision @ k
Esta é a métrica **mais importante para auditores**.
Se o auditor só tem tempo para verificar 50 casos (`k=50`), quantos desses 50 são realmente fraudes?

Isso mede a qualidade do ranking.

In [ ]:
from pyod.utils.utility import precision_n_scores

# Calcula precisão @ n (onde n é o número de outliers reais existentes no dataset)
p_at_n = precision_n_scores(y_test, y_scores)
print(f"Precision @ n: {p_at_n:.2f}")

# Exemplo manual: Precision @ Top 10
df_res = pd.DataFrame({'score': y_scores, 'label': y_test})
df_res = df_res.sort_values('score', ascending=False)

top_10 = df_res.head(10)
acertos_top_10 = top_10['label'].sum()
print(f"Em 10 casos investigados de maior score, encontramos {int(acertos_top_10)} fraudes reais.")

## Conclusão
Metrics como Precision@K são muito mais úteis para o negócio do que acurácia simples, pois refletem a eficiência da força de trabalho da auditoria.